# Setup and compile model

In [2]:
'''setup prediction model using trained weights, first copmile model'''
import midi
import numpy as np

# ''' define rnn model '''
from keras.models import Model
from keras.layers import Input, Dense, LSTM, CuDNNLSTM
from keras import metrics, regularizers

units = 1000

'layer 1 is the encoder'
encoder_inputs = Input(shape=(None, 15))
encoder = LSTM(units, return_state=True)
encoder_outputs, encoder_state_h, encoder_state_c = encoder(encoder_inputs)
'encoder_outputs wont be used'

'layer 2 is the decoder'
decoder_inputs = Input(shape=(None, 15))
decoder = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_h, decoder_state_c = decoder(
    decoder_inputs, initial_state=[encoder_state_h, encoder_state_c])

'layer 3 is a softmax layer for output'
decoder_dense = Dense(15, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', 
              metrics=[metrics.categorical_accuracy])

def map_notes_to_one_octave(note_values):
    output = np.array(note_values)
    for i in range(0, len(note_values)):
        note = note_values[i]
        if(note == 0):
            output[i] = -1
        else:
            output[i] = note%12
    return output


encoder_model = Model(encoder_inputs, [encoder_state_h, encoder_state_c])

decoder_state_input_h = Input(shape=(units,))
decoder_state_input_c = Input(shape=(units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder(
    decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]

decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

'function to take sequence of note values and return another one'
'input_seq is (1, 128, 57)'
'output is (128, 57)'

def decode_sequence(input_seq):
    'run encoder to get the state that will be input for the decoder'
    states_value = encoder_model.predict(input_seq)
    
    'decode starts start of sequence vector'
    target = np.zeros((1, 1, 15))
    target[0, 0, 13] = 1
    
    'decoded_sequence contains notes in the output'
    decoded_sequence = np.zeros((number_of_notes, 15))
        
    'loop generates 64 notes'
    for i in range(0, number_of_notes):
        'running decoder to predict next value given target note and input state'
        output_note, h, c = decoder_model.predict([target] + states_value)
        
        'getting note with highest softmax value'
        note_index = np.argmax(output_note)
        
        'updating output sequence'
        decoded_sequence[i, note_index] = 1 
        
        'setting next target to be the previous note'
        target = np.zeros((1, 1, 15))            
        target[0, 0, note_index] = 1
        'updating the decoder input for next iteration'
        states_value = [h, c]
        
    return decoded_sequence

# Setup functions to handle midi files

In [2]:
def read_midi_file(book_num, fugue_num):
    path = "processed midi files/book_" + str(book_num) + "/fugue_" + str(fugue_num) + ".mid"
    return midi.read_midifile(path)

def get_note_values_from_track(track):
    # 1) go through midi track and find a midi on event
    # 2) find it's position by dividing the total_ticks by size of smallest_note
    # 3) figure out length of track
    # 4) create an array with that length that has the values
    index_and_value = []
    total_ticks = 0
    for i in range(0, len(track)):
        current_event = track[i]
        # in the begining some events are not note events
        if(type(current_event) != midi.events.NoteOnEvent 
           and type(current_event) != midi.events.NoteOffEvent):
            continue
        else:
            total_ticks += current_event.tick
        if(type(current_event) == midi.events.NoteOnEvent):
            index = total_ticks / 6 # always a multiple of 6
            index_and_value.append((index, current_event.data[0]))
            # this will be like (7, 62) meaning note 62 at index 7
        
    # might be possible to do this in a better way
    numberOfNotes = index_and_value[len(index_and_value)-1][0]+1
    note_values = np.zeros(numberOfNotes)
    # this just creates the final array from the (index, value) pairs
    for pair in index_and_value:
        note_values[pair[0]] = pair[1]
    
    return note_values            

def get_X_from_file():
    file_name = 'custom_input.mid'
    X = midi.read_midifile(file_name)
    position = 0
    if(len(X)>=2):
        position = 1
    X = get_note_values_from_track(X[position])
    np.save('custom_input.npy', X)
    return X

def array_of_notes_to_midi(notes, filename):
    # Instantiate a MIDI Pattern (contains a list of tracks)
    pattern = midi.Pattern(resolution=96)
    # Instantiate a MIDI Track (contains a list of MIDI events)
    track = midi.Track()
    # Append the track to the pattern
    pattern.append(track)
    for i in range(0, len(notes)):
        note = notes[i]
        if(note<0):
            note = 0
        # Instantiate a MIDI note on event, append it to the track
        on = midi.NoteOnEvent(tick=0, velocity=70, pitch=note)
        track.append(on)
        # Instantiate a MIDI note off event, append it to the track
        off = midi.NoteOffEvent(tick=6, pitch=note)
        track.append(off)
    # Add the end of track event, append it to the track
    eot = midi.EndOfTrackEvent(tick=1)
    track.append(eot)
    # Print out the pattern
    # print pattern
    # Save the pattern to disk
    midi.write_midifile(filename, pattern)

In [3]:
'''read input sequence and write an output sequence that can later be transformed into midi'''

def note_values_to_one_hot_phrase(note_values):
    note_values = map_notes_to_one_octave(note_values)
    ''' 12 is 0, 13 is start, 14 is end '''
    ''' values are between 0 and 11'''
    one_hot_phrase = np.zeros((number_of_notes+2, 15))
    'start of sequence'
    one_hot_phrase[0][13] = 1 
    for i in range (1, len(note_values)+1):
        note = int(note_values[i-1])
        one_hot_note = np.zeros(15)
        if(note == -1):
            one_hot_note[12] = 1
        else:
            one_hot_note[note] = 1
        one_hot_phrase[i] = one_hot_note
    'end of sequence'
    one_hot_phrase[-1][14] = 1 
        
    return one_hot_phrase

def one_hot_phrase_to_note_values(one_hot_phrase):
    ''' 12 is 0, 13 is start, 14 is end '''
    note_values = np.zeros((number_of_notes+2, 1))
    for i in range(0, len(one_hot_phrase)):
        index, = np.where(one_hot_phrase[i] == 1)
        if(len(index) == 0):
            continue
        if(index[0] == 12):
            note_values[i] = 0 
        elif(index[0] == 13):
            note_values[i] = -1
        elif(index[0] == 14):
            note_values[i] = -2
        else:
            note_values[i] = index[0]
    return note_values

# Start Predictions

In [4]:
'this create a npy file from the midi input which must be name custom_input.midi'
number_of_notes = 64
get_X_from_file()

array([36., 36., 36., 36., 36., 36., 36., 39., 39., 39., 39., 39., 39.,
       39., 39., 40., 40., 40., 40., 40., 40., 40., 40., 40., 40., 40.,
       40., 40., 40., 40., 40., 39., 39., 39., 39., 39., 39., 39., 39.,
       39., 39., 39., 39., 39., 39., 39., 39., 42., 42., 42., 42., 42.,
       42., 42., 42., 42., 42., 42., 42., 42., 42.])

In [3]:
'this creates an output array using the custom_input.npy from previous step'

'load best model'
model.load_weights('lstm_units_1000_batch_64_whole_dataset.h5')

custom_input = np.zeros((1, number_of_notes+2, 15))
custom_output = np.zeros((1, number_of_notes+2, 15))
custom_input[0] = note_values_to_one_hot_phrase(np.load('custom_input.npy'))
custom_output = decode_sequence(custom_input)

print(one_hot_phrase_to_note_values(custom_input[0]).T)
print(one_hot_phrase_to_note_values(custom_output).T)

np.save('custom_output.npy', np.asarray(
    one_hot_phrase_to_note_values(custom_output).T, dtype='int'))

[[-1.  0.  0.  0.  0.  0.  0.  0.  0.  3.  3.  3.  3.  3.  3.  3.  3.  4.
   4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  3.  3.  3.
   3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  6.  6.  6.  6.  6.
   6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6. -2.]]
[[-1.  0. 11.  6.  6.  6.  6.  6.  6. 11.  8.  8.  8.  8.  8.  8.  8. 10.
  10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.
  10. 10. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11.
  11. 11. 11. 11. 11. 11. 11. 11. 11. 11.  0.  0.]]


In [ ]:
'this create the custom_output.midi file from custom_output.npy generated in previous step'
output_of_model = np.load('custom_output.npy')[0]

output_of_model = np.array(output_of_model, dtype='int')

print(output_of_model)
array_of_notes_to_midi(output_of_model[1:-2], 'custom_output.mid')